In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import matplotlib.pyplot as plot
from wordcloud import WordCloud
import string
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv("/kaggle/input/movies-dataset-for-feature-extracion-prediction/movies.csv")
data.head()

In [ ]:
data.info()

In [ ]:
data.describe(include = 'all')

MISSING VALUES

In [ ]:
data.isnull().sum()

Identifying Datatype

In [ ]:
plot.figure()
sns.heatmap(data.isnull(), yticklabels=False, cbar=True)

In [ ]:
numcolumn = data._get_numeric_data().columns.tolist()
print(numcolumn)

In [ ]:
catcolumn = set(data.columns) - set(num_col)
print(catcolumn)

In [ ]:
for i in catcolumn:
    data[i].fillna(data[i].mode()[0], inplace=True)
for i in numcolumn:
    data[i].fillna(data[i].mean(), inplace=True)

In [ ]:
data.isnull().sum()